<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

Clustering with scikit-learn

<br><br></p>

In this notebook, we will learn how to perform k-means lustering using scikit-learn in Python. 

We will use cluster analysis to generate a big picture model of the weather at a local station using a minute-graunlarity data. In this dataset, we have in the order of millions records. How do we create 12 clusters our of them?

**NOTE:** The dataset we will use is in a large CSV file called *minute_weather.csv*. Please download it into the *weather* directory in your *Week-7-MachineLearning* folder. The download link is: https://drive.google.com/open?id=0B8iiZ7pSaSFZb3ItQ1l4LWRMTjg 

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importing the Necessary Libraries<br></p>

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates

%matplotlib inline

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Creating a Pandas DataFrame from a CSV file<br><br></p>


In [2]:
data = pd.read_csv('./weather/minute_weather.csv')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold">Minute Weather Data Description</p>
<br>
The **minute weather dataset** comes from the same source as the daily weather dataset that we used in the decision tree based classifier notebook. The main difference between these two datasets is that the minute weather dataset contains raw sensor measurements captured at one-minute intervals. Daily weather dataset instead contained processed and well curated data. The data is in the file **minute_weather.csv**, which is a comma-separated file.

As with the daily weather data, this data comes from a weather station located in San Diego, California. The weather station is equipped with sensors that capture weather-related measurements such as air temperature, air pressure, and relative humidity. Data was collected for a period of three years, from September 2011 to September 2014, to ensure that sufficient data for different seasons and weather conditions is captured.

Each row in **minute_weather.csv** contains weather data captured for a one-minute interval. Each row, or sample, consists of the following variables:

* **rowID:** 	unique number for each row	(*Unit: NA*)
* **hpwren_timestamp:**	timestamp of measure	(*Unit: year-month-day hour:minute:second*)
* **air_pressure:** air pressure measured at the timestamp	(*Unit: hectopascals*)
* **air_temp:**	air temperature measure at the timestamp	(*Unit: degrees Fahrenheit*)
* **avg_wind_direction:**	wind direction averaged over the minute before the timestamp	(*Unit: degrees, with 0 means coming from the North, and increasing clockwise*)
* **avg_wind_speed:**	wind speed averaged over the minute before the timestamp	(*Unit: meters per second*)
* **max_wind_direction:**	highest wind direction in the minute before the timestamp	(*Unit: degrees, with 0 being North and increasing clockwise*)
* **max_wind_speed:**	highest wind speed in the minute before the timestamp	(*Unit: meters per second*)
* **min_wind_direction:**	smallest wind direction in the minute before the timestamp	(*Unit: degrees, with 0 being North and inceasing clockwise*)
* **min_wind_speed:**	smallest wind speed in the minute before the timestamp	(*Unit: meters per second*)
* **rain_accumulation:**	amount of accumulated rain measured at the timestamp	(*Unit: millimeters*)
* **rain_duration:**	length of time rain has fallen as measured at the timestamp	(*Unit: seconds*)
* **relative_humidity:**	relative humidity measured at the timestamp	(*Unit: percent*)

In [3]:
data.shape

(1587257, 13)

In [4]:
data.head()

rowID     hpwren_timestamp  air_pressure  air_temp  avg_wind_direction  \
0      0  2011-09-10 00:00:49         912.3     64.76                97.0   
1      1  2011-09-10 00:01:49         912.3     63.86               161.0   
2      2  2011-09-10 00:02:49         912.3     64.22                77.0   
3      3  2011-09-10 00:03:49         912.3     64.40                89.0   
4      4  2011-09-10 00:04:49         912.3     64.40               185.0   

   avg_wind_speed  max_wind_direction  max_wind_speed  min_wind_direction  \
0             1.2               106.0             1.6                85.0   
1             0.8               215.0             1.5                43.0   
2             0.7               143.0             1.2               324.0   
3             1.2               112.0             1.6                12.0   
4             0.4               260.0             1.0               100.0   

   min_wind_speed  rain_accumulation  rain_duration  relative_humidity  
0             1.0                NaN            NaN               60.5  
1             0.2                0.0            0.0               39.9  
2             0.3                0.0            0.0               43.0  
3             0.7                0.0            0.0               49.5  
4             0.1                0.0            0.0               58.8

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Data Sampling<br></p>

Lots of rows, so let us sample down by taking every 10th row. <br>


In [5]:
sampled_df = data[(data['rowID'] % 10) == 0]
sampled_df.shape

(158726, 13)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Statistics
<br><br></p>


In [6]:
sampled_df.describe().transpose()

count           mean            std     min       25%  \
rowID               158726.0  793625.000000  458203.937509    0.00  396812.5   
air_pressure        158726.0     916.830161       3.051717  905.00     914.8   
air_temp            158726.0      61.851589      11.833569   31.64      52.7   
avg_wind_direction  158680.0     162.156100      95.278201    0.00      62.0   
avg_wind_speed      158680.0       2.775215       2.057624    0.00       1.3   
max_wind_direction  158680.0     163.462144      92.452139    0.00      68.0   
max_wind_speed      158680.0       3.400558       2.418802    0.10       1.6   
min_wind_direction  158680.0     166.774017      97.441109    0.00      76.0   
min_wind_speed      158680.0       2.134664       1.742113    0.00       0.8   
rain_accumulation   158725.0       0.000318       0.011236    0.00       0.0   
rain_duration       158725.0       0.409627       8.665523    0.00       0.0   
relative_humidity   158726.0      47.609470      26.214409    0.90      24.7   

                          50%         75%         max  
rowID               793625.00  1190437.50  1587250.00  
air_pressure           916.70      918.70      929.50  
air_temp                62.24       70.88       99.50  
avg_wind_direction     182.00      217.00      359.00  
avg_wind_speed           2.20        3.80       31.90  
max_wind_direction     187.00      223.00      359.00  
max_wind_speed           2.70        4.60       36.00  
min_wind_direction     180.00      212.00      359.00  
min_wind_speed           1.60        3.00       31.60  
rain_accumulation        0.00        0.00        3.12  
rain_duration            0.00        0.00     2960.00  
relative_humidity       44.70       68.00       93.00

In [7]:
sampled_df[sampled_df['rain_accumulation'] == 0].shape

(157812, 13)

In [8]:
sampled_df[sampled_df['rain_duration'] == 0].shape

(157237, 13)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Drop all the Rows with Empty rain_duration and rain_accumulation
<br><br></p>


In [9]:
del sampled_df['rain_accumulation']
del sampled_df['rain_duration']

In [10]:
rows_before = sampled_df.shape[0]
sampled_df = sampled_df.dropna()
rows_after = sampled_df.shape[0]

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

How many rows did we drop ?
<br><br></p>


In [11]:
rows_before - rows_after

46

In [12]:
sampled_df.columns

Index(['rowID', 'hpwren_timestamp', 'air_pressure', 'air_temp',
       'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction',
       'max_wind_speed', 'min_wind_direction', 'min_wind_speed',
       'relative_humidity'],
      dtype='object')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Select Features of Interest for Clustering
<br><br></p>


In [13]:
features = ['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction', 
        'max_wind_speed','relative_humidity']

In [14]:
select_df = sampled_df[features]

In [15]:
select_df.columns

Index(['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed',
       'max_wind_direction', 'max_wind_speed', 'relative_humidity'],
      dtype='object')

In [16]:
select_df

air_pressure  air_temp  avg_wind_direction  avg_wind_speed  \
0               912.3     64.76                97.0             1.2   
10              912.3     62.24               144.0             1.2   
20              912.2     63.32               100.0             2.0   
30              912.2     62.60                91.0             2.0   
40              912.2     64.04                81.0             2.6   
...               ...       ...                 ...             ...   
1587210         915.9     75.56               330.0             1.0   
1587220         915.9     75.56               330.0             1.1   
1587230         915.9     75.56               344.0             1.4   
1587240         915.9     75.20               359.0             1.3   
1587250         915.9     74.84                 6.0             1.5   

         max_wind_direction  max_wind_speed  relative_humidity  
0                     106.0             1.6               60.5  
10                    167.0             1.8               38.5  
20                    122.0             2.5               58.3  
30                    103.0             2.4               57.9  
40                     88.0             2.9               57.4  
...                     ...             ...                ...  
1587210               341.0             1.3               47.8  
1587220               341.0             1.4               48.0  
1587230               352.0             1.7               48.0  
1587240                 9.0             1.6               46.3  
1587250                20.0             1.9               46.1  

[158680 rows x 7 columns]

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Scale the Features using StandardScaler
<br><br></p>


In [17]:
X = StandardScaler().fit_transform(select_df)
X

array([[-1.48456281,  0.24544455, -0.68385323, ..., -0.62153592,
        -0.74440309,  0.49233835],
       [-1.48456281,  0.03247142, -0.19055941, ...,  0.03826701,
        -0.66171726, -0.34710804],
       [-1.51733167,  0.12374562, -0.65236639, ..., -0.44847286,
        -0.37231683,  0.40839371],
       ...,
       [-0.30488381,  1.15818654,  1.90856325, ...,  2.0393087 ,
        -0.70306017,  0.01538018],
       [-0.30488381,  1.12776181,  2.06599745, ..., -1.67073075,
        -0.74440309, -0.04948614],
       [-0.30488381,  1.09733708, -1.63895404, ..., -1.55174989,
        -0.62037434, -0.05711747]])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Use k-Means Clustering
<br><br></p>


In [18]:
kmeans = KMeans(n_clusters=12)
model = kmeans.fit(X)
print("model\n", model)

model
 KMeans(n_clusters=12)


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

What are the centers of 12 clusters we formed ?
<br><br></p>


In [19]:
centers = model.cluster_centers_
centers

array([[ 0.12985916,  0.84567136,  1.41090187, -0.63878871,  1.67519544,
        -0.58970443, -0.71279841],
       [ 0.05765817, -0.78594055, -1.19626637, -0.57096053, -1.04249471,
        -0.58558532,  0.87686716],
       [-0.68072565,  0.56483376,  0.18011088, -0.58691672,  0.35020759,
        -0.60020308, -0.13189936],
       [ 1.36625812, -0.08070507, -1.20794736, -0.04948948, -1.07676327,
        -0.02914741, -0.97757017],
       [-0.213469  ,  0.63531582,  0.40843772,  0.7335086 ,  0.51657496,
         0.67159353, -0.15104393],
       [ 0.21271454, -0.99883942,  0.64965851, -0.54583654,  0.83837156,
        -0.52901845,  1.17453366],
       [-1.18280623, -0.86356618,  0.44710461,  1.99261576,  0.53800623,
         1.95310839,  0.90161562],
       [ 0.75908451,  0.39604498,  0.29356794, -0.52666643,  0.48085924,
        -0.53236644, -0.76588139],
       [-0.16303156,  0.86389641, -1.30985965, -0.59023723, -1.16532501,
        -0.60551527, -0.64198616],
       [ 0.23399236,  0.3200

<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

Plots
<br><br></p>


Let us first create some utility functions which will help us in plotting graphs:

In [20]:
# Function that creates a DataFrame with a column for Cluster Number

def pd_centers(featuresUsed, centers):
	colNames = list(featuresUsed)
	colNames.append('prediction')

	# Zip with a column called 'prediction' (index)
	Z = [np.append(A, index) for index, A in enumerate(centers)]

	# Convert to pandas data frame for plotting
	P = pd.DataFrame(Z, columns=colNames)
	P['prediction'] = P['prediction'].astype(int)
	return P

In [23]:
# Function that creates Parallel Plots

def parallel_plot(data):
	my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k']), None, len(data)))
	plt.figure(figsize=(15,8)).gca().axes.set_ylim([-3,+3])
	parallel_coordinates(data, 'prediction', color = my_colors, marker='o')

In [21]:
P = pd_centers(features, centers)
P

air_pressure  air_temp  avg_wind_direction  avg_wind_speed  \
0       0.129859  0.845671            1.410902       -0.638789   
1       0.057658 -0.785941           -1.196266       -0.570961   
2      -0.680726  0.564834            0.180111       -0.586917   
3       1.366258 -0.080705           -1.207947       -0.049489   
4      -0.213469  0.635316            0.408438        0.733509   
5       0.212715 -0.998839            0.649659       -0.545837   
6      -1.182806 -0.863566            0.447105        1.992616   
7       0.759085  0.396045            0.293568       -0.526666   
8      -0.163032  0.863896           -1.309860       -0.590237   
9       0.233992  0.320021            1.887919       -0.651769   
10     -0.842368 -1.200905            0.376460        0.395155   
11      1.190472 -0.255122           -1.154948        2.123766   

    max_wind_direction  max_wind_speed  relative_humidity  prediction  
0             1.675195       -0.589704          -0.712798           0  
1            -1.042495       -0.585585           0.876867           1  
2             0.350208       -0.600203          -0.131899           2  
3            -1.076763       -0.029147          -0.977570           3  
4             0.516575        0.671594          -0.151044           4  
5             0.838372       -0.529018           1.174534           5  
6             0.538006        1.953108           0.901616           6  
7             0.480859       -0.532366          -0.765881           7  
8            -1.165325       -0.605515          -0.641986           8  
9            -1.551747       -0.576601          -0.283582           9  
10            0.474698        0.382584           1.357163          10  
11           -1.053479        2.240525          -1.134616          11

# Dry Days

In [24]:
parallel_plot(P[P['relative_humidity'] < -0.5])

# Warm Days

In [25]:
parallel_plot(P[P['air_temp'] > 0.5])

# Cool Days

In [26]:
parallel_plot(P[(P['relative_humidity'] > 0.5) & (P['air_temp'] < 0.5)])